In [1]:
include("src\\ImportanceSampling.jl")
using Distributions
using ImportanceSampling

In [2]:
p = Weibull(2, 4)
q = Exponential(2)
f!(r, x) = r[1:2] = Float64[x[1], x[1]^2]

is = ImportanceSampler(f!, 2, q, p=p)

N = 5000
X = rand(q, N)
;

In [4]:
t = zeros(2)
m = sum(f!(t, X[i]).*pdf(p, X[i])/pdf.(q, X[i]) for i in 1:N) ./ N
isapprox(mean(is), m)

false

In [5]:
mean(is) - [mean(p), var(p) + mean(p)^2]

2-element Array{Float64,1}:
 NaN
 NaN

# MV

In [84]:
p = MvNormal(2.0*ones(2), eye(2))
q = MvTDist(1, zeros(2), eye(2)*5)
f!(r, x) = r[:] = vcat(x, x.^2)

is = ImportanceSampler(f!, 4, q, p=p)

N = 5000
X = rand(q, N)
;

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] sqrt(::Array{Float64,2}) at .\deprecated.jl:57
 [3] _rand!(::Distributions.GenericMvTDist{Float64,PDMats.PDMat{Float64,Array{Float64,2}}}, ::Array{Float64,2}) at C:\Users\Hamza El-Saawy\.julia\v0.6\Distributions\src\multivariate\mvtdist.jl:176
 [4] rand(::Distributions.GenericMvTDist{Float64,PDMats.PDMat{Float64,Array{Float64,2}}}, ::Int64) at C:\Users\Hamza El-Saawy\.julia\v0.6\Distributions\src\multivariates.jl:50
 [5] include_string(::String, ::String) at .\loading.jl:515
 [6] include_string(::Module, ::String, ::String) at C:\Users\Hamza El-Saawy\.julia\v0.6\Compat\src\Compat.jl:71
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Hamza El-Saawy\.julia\v0.6\IJulia\src\execute_request.jl:158
 [8] (::Compat.#inner#17{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\Hamza El-Saawy\.julia\v0.6\Compat\src\Compat.jl:385
 [9] eventloop(::ZMQ.Socket) at C:\Users\Hamza El-Saaw

In [48]:
update!(is, X=X)

ImportanceSampling.Diagnostic(5000, 5100.831515511719, 39068.02775873413, 405048.3541831882, 4.76868615717365e6)

In [58]:
t = zeros(4)
m = sum(f!(t, X[:, i]).*pdf(p, X[:, i])/pdf(q, X[:, i]) for i in 1:N) ./ N
isapprox(mean(is), m)

4-element Array{Float64,1}:
 2.07596
 2.03016
 5.25755
 5.05987

In [65]:
mean(is)

4-element Array{Float64,1}:
 2.07596
 2.03016
 5.25755
 5.05987

In [62]:
mean(is) - vcat(mean(p), var(p) + mean(p).^2)

4-element Array{Float64,1}:
 0.0759634
 0.0301567
 0.257546 
 0.0598705

# exponential tilting

$$ w(x) = p(x, \theta_0)/p(x, \theta) $$

## normal

In [156]:
function gen_w_normal(μ0, μ, Σ)
    invΣ = inv(Σ)
    dη = invΣ*(μ0-μ)
    C = exp(1/2*( dot(μ, invΣ*μ) - dot(μ0, invΣ*μ0) ))
    return x -> C*exp(dot(x, dη))
end

gen_w_normal (generic function with 2 methods)

In [161]:
f!(r, x) = r[1:2] = Float64[x[1], x[1]^2]

m0s = linspace(-5, 5, 11)
m = 1.0
q = Normal()
ws = [gen_w_normal(m0, 0, 1) for m0 in m0s]
iss = [ImportanceSampler(f!, 1, q, w=w) for w in ws]
;

In [167]:
xor(0, 0)

0

In [165]:
map(is -> update!(is; niters=100_000, batchsize=1000), iss);

LoadError: [91mboth F or W must be provided if X is not[39m